In [1]:
import sys
if 'google.colab' in sys.modules:  # colab特有の処理_2回目以降
  # Google Driveのマウント
  from google.colab import drive
  drive.mount('/content/drive')

  # ライブラリのパス指定
  sys.path.append('/content/drive/MyDrive/Colab_Files/kaggle/commonlit/XX_modules')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# データセットをDriveから取得
!mkdir -p 'input'
!mkdir -p 'clrp-pre-trained'

!cp -r '/content/drive/MyDrive/Colab_Files/kaggle/commonlit/00_input/commonlitreadabilityprize/' '/content/input'
!cp -r '/content/drive/MyDrive/Colab_Files/kaggle/commonlit/97_pre_trained/clrp_pretrained_manish_epoch5/pre-trained-roberta/clrp_roberta_large/' '/content/clrp-pre-trained'

In [3]:
from pathlib import Path

# input
if 'kaggle_web_client' in sys.modules:  # kaggle環境
    DATA_DIR = Path('../input/commonlitreadabilityprize/')

elif 'google.colab' in sys.modules: # Colab環境
    DATA_DIR = Path('/content/input/commonlitreadabilityprize')

else:
    DATA_DIR = Path('../00_input/commonlitreadabilityprize/')

In [4]:
from pathlib import Path

# tokenizer
if 'kaggle_web_client' in sys.modules:  # kaggle環境
    TOKENIZER_DIR = '../input/roberta-transformers-pytorch/roberta-large'
elif 'google.colab' in sys.modules: # Colab環境
    TOKENIZER_DIR = '/content/clrp-pre-trained/clrp_roberta_large' # 仮で、毎回DLする想定のモデル名を指定。あとで変更予定。
else:
    TOKENIZER_DIR = 'roberta-large'

In [5]:
from pathlib import Path

# pre-trained model
if 'kaggle_web_client' in sys.modules:  # kaggle環境
    PRE_TRAINED_MODEL_DIR = '../input/roberta-transformers-pytorch/roberta-large'
elif 'google.colab' in sys.modules: # Colab環境
    PRE_TRAINED_MODEL_DIR = '/content/clrp-pre-trained/clrp_roberta_large' # 仮で、毎回DLする想定のモデル名を指定。あとで変更予定。
else:
    PRE_TRAINED_MODEL_DIR = 'roberta-large'

In [6]:
UPLOAD_DIR = Path('/content/model')
EX_NO = '055-054-train-05'  # 実験番号などを入れる、folderのpathにする
USERID = 'calpis10000'

In [7]:
import subprocess
import shlex

def gpuinfo():
    """
    Returns size of total GPU RAM and used GPU RAM.

    Parameters
    ----------
    None

    Returns
    -------
    info : dict
        Total GPU RAM in integer for key 'total_MiB'.
        Used GPU RAM in integer for key 'used_MiB'.
    """

    command = 'nvidia-smi -q -d MEMORY | sed -n "/FB Memory Usage/,/Free/p" | sed -e "1d" -e "4d" -e "s/ MiB//g" | cut -d ":" -f 2 | cut -c2-'
    commands = [shlex.split(part) for part in command.split(' | ')]
    for i, cmd in enumerate(commands):
        if i==0:
            res = subprocess.Popen(cmd, stdout=subprocess.PIPE)
        else:
            res = subprocess.Popen(cmd, stdin=res.stdout, stdout=subprocess.PIPE)
    total, used = map(int, res.communicate()[0].decode('utf-8').strip().split('\n'))
    info = {'total_MiB':total, 'used_MiB':used}
    return info


# Overview
This nb is based on copy from https://www.kaggle.com/andretugan/lightweight-roberta-solution-in-pytorch .

Acknowledgments(from base nb): 
some ideas were taken from kernels by [Torch](https://www.kaggle.com/rhtsingh) and [Maunish](https://www.kaggle.com/maunish).

In [8]:
import os
import math
import random
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import AdamW # optimizer
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoConfig
from transformers import get_cosine_schedule_with_warmup # scheduler
from pytorch_memlab import profile
import pytorch_memlab
from pytorch_memlab import MemReporter

from sklearn.model_selection import KFold, StratifiedKFold

import pickle
import gc
gc.enable()

In [9]:
NUM_FOLDS = 5 # K Fold
NUM_EPOCHS = 5 # Epochs
BATCH_SIZE = 8 # Batch Size
MAX_LEN = 248 # ベクトル長
TFIDF_MAX_FEAT = 1024

EVAL_SCHEDULE = [(0.55, 64), (-1., 32)] # schedulerの何らかの設定？
ROBERTA_PATH = PRE_TRAINED_MODEL_DIR # roberta pre-trainedモデル(モデルとして指定)
TOKENIZER_PATH = TOKENIZER_DIR # roberta pre-trainedモデル(Tokenizerとして指定)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu" # cudaがなければcpuを使えばいいじゃない

In [10]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)

    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

    torch.backends.cudnn.deterministic = True# cudnnによる最適化で結果が変わらないためのおまじない 

In [11]:
# read train_df(kfold)
train_kf_df = pd.read_csv(DATA_DIR/"train_kfold.csv")

In [12]:
# tokenizerを指定
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

# Dataset

In [13]:
# 前処理用
import string
import re
# ローカルの場合、stopwordsをダウンロード
import nltk
if 'kaggle_web_client' in sys.modules:  # kaggle環境
    pass
else:
    import nltk
    nltk.download('stopwords')
    nltk.download('averaged_perceptron_tagger')
    os.listdir(os.path.expanduser('~/nltk_data/corpora/stopwords/'))

# テキスト前処理
# https://www.kaggle.com/alaasedeeq/commonlit-readability-eda

#filtering the unwanted symbols, spaces, ....etc
to_replace_by_space = re.compile('[/(){}\[\]|@,;]')
punctuation = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
bad_symbols = re.compile('[^0-9a-z #+_]')
stopwords = set(nltk.corpus.stopwords.words('english'))

def text_prepare(text):
    '''
    text: a string
    returna modified version of the string
    '''
    text = text.lower() # lowercase text
    text = re.sub(punctuation, '',text)
    text = re.sub(to_replace_by_space, " ", text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(bad_symbols, "", text)         # delete symbols which are in BAD_SYMBOLS_RE from text
    text = " ".join([word for word in text.split(" ") if word not in stopwords]) # delete stopwords from text
    text = re.sub(' +', ' ', text)
    return text

def text_normalization(s:pd.Series):
    x = s.apply(text_prepare)
    return x

# Counterオブジェクトを取得
def get_counter(text:str):
    text_list = [wrd for wrd in text.split(" ") if wrd not in ('', '\n')]
    counter = collections.Counter(text_list)
    return counter


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

# ベースとなる継承元のクラス
class BaseBlock(object):
    def fit(self, input_df, y=None):
        return self.transform(input_df)
    def transform(self, input_df):
        raise NotImplementedError()

class CountVectorizerSimpleBlock(BaseBlock):
    """CountVectorizerのベクトルをそのまま返す block"""
    def __init__(self, column: str, master_df=None, max_features=50, ngram_range=(1,1)):
        """
        args:
            column: str
                変換対象のカラム名
        """
        self.column = column
        self.master_df=master_df
        self.max_features=max_features
        self.ngram_range=ngram_range
        self.param_prefix = f"col={column}_feats={max_features}_ngram={''.join([str(i) for i in self.ngram_range])}"

    def preprocess(self, input_df):
        x = text_normalization(input_df[self.column])
        return x

    def fit(self, 
            input_df, 
            y=None
           ):
        master_df = input_df if self.master_df is None else self.master_df
        text = self.preprocess(master_df)
        self.vectorizer_ = CountVectorizer(max_features=self.max_features, ngram_range=self.ngram_range)

        self.vectorizer_.fit(text)
        self.prefix = 'countvec'
        return self.transform(input_df)

    def transform(self, input_df):
        text = self.preprocess(input_df)
        z = self.vectorizer_.transform(text)

        out_df = pd.DataFrame(z.toarray())
        out_df.columns = self.vectorizer_.get_feature_names()
        return out_df.add_prefix(f'{self.prefix}_')

In [15]:
class LitDataset(Dataset):
    def __init__(self, df, vectorizer_, inference_only=False):
        super().__init__()

        self.df = df        
        self.inference_only = inference_only # Testデータ用フラグ
        self.text = df.excerpt.tolist() # 分析対象カラムをlistにする。(分かち書きではなく、Seriesをlistへ変換するような処理)
        #self.text = [text.replace("\n", " ") for text in self.text] # 単語単位で分かち書きする場合
        self.text_len = text_normalization(df.excerpt).map(lambda x: [0 if i >= len(x.split(' ')) else len(x.split(' ')[i]) for i in range(132)])
        self.vectorizer_ = vectorizer_ # fit済みのものを入力する。（クラス内ではtransformのみ行う）
        self.vectorizer_val = vectorizer_.transform(df).values

        if not self.inference_only:
            self.target = torch.tensor(df.target.values, dtype=torch.float32) # trainのみ、targetをtensorに変換
            self.standard_error = torch.tensor(df.standard_error.values, dtype=torch.float32) 

        self.encoded = tokenizer.batch_encode_plus( # textをtokenize
            self.text,
            padding = 'max_length',            
            max_length = MAX_LEN,
            truncation = True, # 最大長を超える文字は切り捨て
            return_attention_mask=True
        )        
 

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index): # 変換結果を返す
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        input_len = torch.tensor(self.text_len.iloc[index], dtype=torch.float32)
        input_bow = torch.tensor(self.vectorizer_val[index, :], dtype=torch.float32)

        if self.inference_only:
            return (input_ids, attention_mask, input_len, input_tfidf)            
        else:
            target = self.target[index]
            standard_error = self.standard_error[index]
            return (input_ids, attention_mask, input_len, input_bow, target, standard_error)

# Model
The model is inspired by the one from [Maunish](https://www.kaggle.com/maunish/clrp-roberta-svm).

In [16]:
class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH) # pretrainedからconfigを読み込み
        config.update({"output_hidden_states":True, # config更新: embedding層を抽出
                       "hidden_dropout_prob": 0.0, # config更新: dropoutしない
                       "layer_norm_eps": 1e-7}) # config更新: layer normalizationのepsilon                      
        
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config) # cpuで処理する
            
        self.attention = nn.Sequential(# attentionレイヤー            
            nn.Linear(config.hidden_size, 512),      
            nn.Tanh(),                       
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )

        self.mlp_wordlen = nn.Sequential(
            nn.Linear(132, 64),
            nn.ReLU(),
            nn.Linear(64, 64)
        )

        self.mlp_tfidf = nn.Sequential(
            nn.Linear(TFIDF_MAX_FEAT, 192),
            nn.ReLU(),
            nn.Linear(192, 192),
            nn.ReLU(),
            nn.Linear(192, 64)
        )

        self.regressor = nn.Sequential( # 出力レイヤー                    
            nn.Linear(config.hidden_size + 64 + 64, 2)                        
        )

    def forward(self, input_ids, attention_mask, input_len, input_tfidf):
        roberta_output = self.roberta(input_ids=input_ids, # robertaに入力データを流し、出力としてrobertaモデル(layerの複合体)を得る
                                      attention_mask=attention_mask)     
        # attention_pooling
        last_hidden_state = roberta_output.hidden_states[-1] # robertaモデルの最後のlayerを得る
        weights = self.attention(last_hidden_state) # robertaの最後のlayerをattentionへ入力し、出力として重みを得る                
        context_vector = torch.sum(weights * last_hidden_state, dim=1) # 重み×最後の層を足し合わせて文書ベクトルとする。

        # word_length_conv1d
        #input_chnl = input_len.unsqueeze(1)
        #conv1_layers = self.conv1_layers(input_chnl)
        #conv1_layers_v = conv1_layers.view(conv1_layers.size(0),-1)

        # mlm_layers
        mlp_wordlen = self.mlp_wordlen(input_len)
        mlp_tfidf = self.mlp_tfidf(input_tfidf)

        # https://www.kaggle.com/rhtsingh/utilizing-transformer-representations-efficiently
        # last_hidden_state = roberta_output[0]
        # input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        # sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        # sum_mask = input_mask_expanded.sum(1)
        # sum_mask = torch.clamp(sum_mask, min=1e-9)
        # mean_embeddings = sum_embeddings / sum_mask

        # concat_embeddings
        cat_embeddings = torch.cat([context_vector, mlp_wordlen, mlp_tfidf], dim=1)        
        return self.regressor(cat_embeddings) # 文書ベクトルを線形層に入力し、targetを出力する
        
        # Now we reduce the context vector to the prediction score.
        #return self.regressor(mean_embeddings) # 文書ベクトルを線形層に入力し、targetを出力する

In [17]:
# 評価指標(MSE)の計算。最終的に、ルートしてRMSEにすると思われる。
def eval_mse(model, data_loader):
    """Evaluates the mean squared error of the |model| on |data_loader|"""
    model.eval() # evalモードを選択。Batch Normとかdropoutをしなくなる           
    mse_mean_sum = 0
    mse_std_sum = 0

    with torch.no_grad(): # 勾配の計算をしないBlock
        for batch_num, (input_ids, attention_mask, input_len, input_tfidf, target, standard_error) in enumerate(data_loader): # data_loaderからinput, attentin_mask, targetをbatchごとに取り出す
            input_ids = input_ids.to(DEVICE)   
            attention_mask = attention_mask.to(DEVICE)  
            input_len = input_len.to(DEVICE) 
            input_tfidf = input_tfidf.to(DEVICE)
            target = target.to(DEVICE)      
            standard_error = standard_error.to(DEVICE) 
            
            output = model(input_ids, attention_mask, input_len, input_tfidf) # 取得した値をモデルへ入力し、出力として予測値を得る。

            mse_mean_sum += nn.MSELoss(reduction="sum")(output[:,0].flatten(), target).item() # 誤差の合計を得る(Batchごとに計算した誤差を足し上げる)
            mse_std_sum += nn.MSELoss(reduction="sum")(output[:,1].flatten(), target).item() # 誤差の合計を得る(Batchごとに計算した誤差を足し上げる)

    del input_ids
    del attention_mask
    del target
    del input_len
    del input_tfidf

    mse_mean_result = mse_mean_sum / len(data_loader.dataset)
    mse_std_result = mse_std_sum / len(data_loader.dataset)
  
    return mse_mean_result, mse_std_result # 誤差の合計をdataset長で除し、mseを取得＆返す

In [18]:
# 推論結果を返す
def predict(model, data_loader):
    """Returns an np.array with predictions of the |model| on |data_loader|"""
    model.eval() # evalモード(dropout, batch_normしない)

    result = np.zeros(len(data_loader.dataset)) # 結果をdataset長のzero配列として用意
    index = 0
    
    with torch.no_grad(): # 勾配の計算をしないblock(inputすると、現状の重みによる推論結果を返す)
        for batch_num, (input_ids, attention_mask, input_len, input_tfidf) in enumerate(data_loader): # data_loaderからbatchごとにinputを得る
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)
            input_len = input_len.to(DEVICE)
            input_tfidf = input_tfidf.to(DEVICE)
                        
            output = model(input_ids, attention_mask, input_len, input_tfidf) # modelにinputを入力し、予測結果を得る。

            result[index : index + output[:,0].shape[0]] = output[:,0].flatten().to("cpu") # result[index ~ predの長さ]へ、予測結果を格納
            index += output.shape[0] # indexを更新

    return result # 全batchで推論が終わったら、結果を返す

In [19]:
# 学習
def train(model, # モデル
          model_path, # モデルのアウトプット先
          train_loader, # train-setのdata_loader
          val_loader, # valid-setのdata_loader
          optimizer, # optimizer
          scheduler=None, # scheduler, デフォルトはNone
          num_epochs=NUM_EPOCHS # epoch数、notebook冒頭で指定した値
         ):    
    
    best_val_rmse = None
    best_epoch = 0
    step = 0
    last_eval_step = 0
    eval_period = EVAL_SCHEDULE[0][1] # eval期間(って何？) 冒頭で決めたEVAL_SCHEDULEの最初のtupleの[1]を取得

    start = time.time() # 時間計測用

    for epoch in range(num_epochs): # 指定したEpoch数だけ繰り返し
        val_rmse = None         

        for batch_num, (input_ids, attention_mask, input_len, input_tfidf, target, standard_error) in enumerate(train_loader): # train_loaderからinput, targetを取得
            input_ids = input_ids.to(DEVICE) # inputをDEVICEへ突っ込む
            attention_mask = attention_mask.to(DEVICE)   
            input_len = input_len.to(DEVICE)
            input_tfidf = input_tfidf.to(DEVICE)
            target = target.to(DEVICE)
            standard_error = standard_error.to(DEVICE)  

            optimizer.zero_grad() # 勾配を初期化            
            model.train() # 学習モード開始

            # https://www.kaggle.com/c/commonlitreadabilityprize/discussion/239421
            output = model(input_ids, attention_mask, input_len, input_tfidf) # input,attention_maskを入力し、予測結果を得る
            p = torch.distributions.Normal(output[:,0], torch.sqrt(output[:,1]**2))
            q = torch.distributions.Normal(target, standard_error)
            kl_vector = torch.distributions.kl_divergence(p, q)
            loss = kl_vector.mean()

            loss.backward() # 誤差逆伝播法により勾配を得る
            optimizer.step() # 重みを更新する

            if scheduler:
                scheduler.step() # schedulerが与えられた場合は、schedulerの学習率更新
            
            if step >= last_eval_step + eval_period: # batchを回すごとにstepを増やしていって、「前回evalしたstep + eval_period(16)」を超えたら実行。
                # Evaluate the model on val_loader.
                elapsed_seconds = time.time() - start # 経過時間
                num_steps = step - last_eval_step # 経過ステップ数
                print(f"\n{num_steps} steps took {elapsed_seconds:0.3} seconds")
                last_eval_step = step # 前回stepの更新
                
                # valid-setによるrmse計算
                train_mean_mse = nn.MSELoss(reduction="mean")(output[:,0].flatten(), target) 
                train_std_mse = nn.MSELoss(reduction="mean")(torch.sqrt(output[:,1]**2).flatten(), standard_error) 

                train_mean_rmse = math.sqrt(train_mean_mse)
                train_std_rmse = math.sqrt(train_std_mse)

                val_mean_mse, val_std_mse = eval_mse(model, val_loader)
                val_mean_rmse = math.sqrt(val_mean_mse)                            
                val_std_rmse = math.sqrt(val_std_mse)                            

                print(f"Epoch: {epoch} batch_num: {batch_num}")
                print(f"train_rmse_target: {train_mean_rmse:0.4}",
                      f"train_rmse_stderror: {train_std_rmse:0.4}",
                      f"train_kl_div: {loss:0.4}",
                      )
                print(f"val_rmse_target: {val_mean_rmse:0.4}",
                      f"val_rmse_stderror: {val_std_rmse:0.4}"
                      )

                for rmse, period in EVAL_SCHEDULE: # eval_periodをvalid-rmseで切り替える処理
                    if val_mean_rmse >= rmse: # valid rmseをEVAL_SCHEDULEと比較し、0項 > valid rmseとなるまで回す : EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
                        eval_period = period # eval_periodを更新
                        break                               

                if not best_val_rmse or val_mean_rmse < best_val_rmse: # 初回(best_val_rmse==None), またはbest_val_rmseを更新したらモデルを保存する
                    best_val_rmse = val_mean_rmse
                    best_epoch = epoch
                    torch.save(model.state_dict(), model_path) # 最高の自分を保存
                    print(f"New best_val_rmse: {best_val_rmse:0.4}")
                else:       
                    print(f"Still best_val_rmse: {best_val_rmse:0.4}", # 更新されない場合は、元のスコアを表示
                          f"(from epoch {best_epoch})")      
                                                  
                start = time.time()
            
            # batchごとにメモリ解放
            del input_ids
            del attention_mask
            del target
            torch.cuda.empty_cache()                                            
            step += 1
    
    return best_val_rmse

In [20]:
# optimizerの作成
def create_optimizer(model):
    parameters = []

    named_parameters = list(model.named_parameters()) # モデルパラメータの取得
    roberta_parameters = list(model.roberta.named_parameters())[:-2] # パラメータをroberta用、attention用、regressor用に格納。(直接引っ張ってくる形式に変更)

    attention_parameters = list(model.attention.named_parameters())
    attention_group = [{'params': params, 'lr': 2e-5} for (name, params) in attention_parameters] # attention用パラメータをリストとして取得
    parameters += attention_group

    #norm_parameters = list(model.layer_norm.named_parameters())
    #norm_group = [{'params': params, 'lr': 2e-5} for (name, params) in norm_parameters]
    #parameters += norm_group

    mlp_wrd_parameters = list(model.mlp_wordlen.named_parameters())
    mlp_wrd_group = [{'params': params, 'lr': 2e-5} for (name, params) in mlp_wrd_parameters] # reg用パラメータをリストとして取得
    parameters += mlp_wrd_group

    mlp_tfidf_parameters = list(model.mlp_tfidf.named_parameters())
    mlp_tfidf_group = [{'params': params, 'lr': 2e-5} for (name, params) in mlp_tfidf_parameters] # reg用パラメータをリストとして取得
    parameters += mlp_tfidf_group

    regressor_parameters = list(model.regressor.named_parameters())
    regressor_group = [{'params': params, 'lr': 2e-5} for (name, params) in regressor_parameters] # reg用パラメータをリストとして取得
    parameters += regressor_group

    for layer_num, (name, params) in enumerate(roberta_parameters): # レイヤーごとにname, paramsを取得していろんな処理
        weight_decay = 0.0 if "bias" in name else 0.01

        lr = 8e-6

        if layer_num >= 69:        
            lr = 2e-5

        if layer_num >= 133:
            lr = 4e-5

        parameters.append({"params": params,
                           "weight_decay": weight_decay,
                           "lr": lr})

    return AdamW(parameters) # 最終的に、AdamWにパラメータを入力する。


In [21]:
# https://www.kaggle.com/abhishek/step-1-create-folds
def create_folds(data, num_splits, SEED, return_df=False):
    # we create a new column called kfold and fill it with -1
    data["kfold"] = -1
    
    # the next step is to randomize the rows of the data
    data = data.sample(frac=1).reset_index(drop=True)

    # calculate number of bins by Sturge's rule
    # I take the floor of the value, you can also
    # just round it
    num_bins = int(np.floor(1 + np.log2(len(data))))
    
    # bin targets
    data.loc[:, "bins_tg"] = pd.cut(
        data["target"], bins=num_bins, labels=False
    ).map(lambda x: str(x))

    # bin standard_error
    data.loc[:, "bins_std"] = pd.cut(
        data["standard_error"], bins=num_bins, labels=False
    )

    # bins
    data.loc[:, "bins"] = data['bins_tg'].map(lambda x: str(x)) + data['bins_std'].map(lambda x: str(x))

    # initiate the kfold class from model_selection module
    kf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)

    # note that, instead of targets, we use bins!
    if return_df:
      for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
      return data
    else:
      return kf.split(X=data, y=data.bins.values)

In [22]:
def train_and_save_model(train_indices, val_indices, model_path, feat_suffix):
    tfidf = CountVectorizerSimpleBlock('excerpt', max_features=TFIDF_MAX_FEAT)
    df_tfidf = tfidf.fit(train_kf_df.loc[train_indices]) # TODO: fit_transformみたいなメソッド作って、fitではdfが返らないようにする。

    with open(f'TFIDF_{feat_suffix}', 'wb') as f:
      pickle.dump(tfidf, f)

    train_dataset = LitDataset(train_kf_df.loc[train_indices], tfidf) # train, validのDataset
    val_dataset = LitDataset(train_kf_df.loc[val_indices], tfidf)
        
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              drop_last=True, shuffle=True, num_workers=2) # train, validのDataLoader
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,
                            drop_last=False, shuffle=False, num_workers=2)    

    model = LitModel().to(DEVICE) # modelをDEVICEへぶち込む
    optimizer = create_optimizer(model) # optimizerをモデルから作成
    scheduler = get_cosine_schedule_with_warmup( # schedulerを作成
        optimizer,
        num_training_steps=NUM_EPOCHS * len(train_loader),
        num_warmup_steps=50)    
    rmse = train(model, model_path, train_loader, val_loader, optimizer, scheduler=scheduler)

    del train_dataset
    del val_dataset
    del train_loader
    del val_loader
    del model
    del optimizer
    del scheduler
    gc.collect() 
    torch.cuda.empty_cache()
    return rmse

In [23]:
# 実行処理。 KFold & 学習
SEED = 1000
list_val_rmse = []

for fold in sorted(train_kf_df['kfold'].unique()):
    print(f"\nFold {fold + 1}/{NUM_FOLDS}")
    print(gpuinfo())
    model_path = f"model_{fold + 1}.pth" # model_fold数_.pth
    set_random_seed(SEED + fold) # SEEDはfold別に変わるようにする
    feat_suffix = f"{fold + 1}.pkl" 

    train_indices = (train_kf_df['kfold'] != fold)
    val_indices = (train_kf_df['kfold'] == fold)
    list_val_rmse.append(train_and_save_model(train_indices, val_indices, model_path, feat_suffix))
    print("\nPerformance estimates:")
    print(list_val_rmse)
    print("Mean:", np.array(list_val_rmse).mean())
    print(gpuinfo())


Fold 1/5
{'total_MiB': 16280, 'used_MiB': 2}


Some weights of the model checkpoint at /content/clrp-pre-trained/clrp_roberta_large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /content/clrp-pre-trained/clrp_roberta_large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN thi


64 steps took 64.1 seconds
Epoch: 0 batch_num: 64
train_rmse_target: 0.5086 train_rmse_stderror: 0.06856 train_kl_div: 0.5348
val_rmse_target: 0.6894 val_rmse_stderror: 1.125
New best_val_rmse: 0.6894

64 steps took 63.1 seconds
Epoch: 0 batch_num: 128
train_rmse_target: 0.6224 train_rmse_stderror: 0.1118 train_kl_div: 0.8163
val_rmse_target: 0.62 val_rmse_stderror: 1.151
New best_val_rmse: 0.62

64 steps took 63.1 seconds
Epoch: 0 batch_num: 192
train_rmse_target: 0.5837 train_rmse_stderror: 0.1028 train_kl_div: 0.5881
val_rmse_target: 0.6123 val_rmse_stderror: 1.138
New best_val_rmse: 0.6123

64 steps took 63.1 seconds
Epoch: 0 batch_num: 256
train_rmse_target: 0.6882 train_rmse_stderror: 0.05332 train_kl_div: 1.07
val_rmse_target: 0.5763 val_rmse_stderror: 1.151
New best_val_rmse: 0.5763

64 steps took 63.4 seconds
Epoch: 1 batch_num: 37
train_rmse_target: 0.732 train_rmse_stderror: 0.1026 train_kl_div: 0.9483
val_rmse_target: 0.5517 val_rmse_stderror: 1.137
New best_val_rmse: 0.55

Some weights of the model checkpoint at /content/clrp-pre-trained/clrp_roberta_large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /content/clrp-pre-trained/clrp_roberta_large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN thi


64 steps took 63.7 seconds
Epoch: 0 batch_num: 64
train_rmse_target: 1.293 train_rmse_stderror: 0.1586 train_kl_div: 3.028
val_rmse_target: 0.9286 val_rmse_stderror: 1.674
New best_val_rmse: 0.9286

64 steps took 63.0 seconds
Epoch: 0 batch_num: 128
train_rmse_target: 0.7168 train_rmse_stderror: 0.06818 train_kl_div: 0.8226
val_rmse_target: 0.6403 val_rmse_stderror: 1.823
New best_val_rmse: 0.6403

64 steps took 63.0 seconds
Epoch: 0 batch_num: 192
train_rmse_target: 0.5957 train_rmse_stderror: 0.1174 train_kl_div: 0.7422
val_rmse_target: 0.6063 val_rmse_stderror: 1.76
New best_val_rmse: 0.6063

64 steps took 63.0 seconds
Epoch: 0 batch_num: 256
train_rmse_target: 0.9499 train_rmse_stderror: 0.1154 train_kl_div: 1.628
val_rmse_target: 0.6337 val_rmse_stderror: 1.735
Still best_val_rmse: 0.6063 (from epoch 0)

64 steps took 63.2 seconds
Epoch: 1 batch_num: 37
train_rmse_target: 0.4412 train_rmse_stderror: 0.1134 train_kl_div: 0.4848
val_rmse_target: 0.618 val_rmse_stderror: 1.75
Still 

...


64 steps took 63.7 seconds
Epoch: 0 batch_num: 64
train_rmse_target: 0.7889 train_rmse_stderror: 0.05908 train_kl_div: 1.34
val_rmse_target: 0.7027 val_rmse_stderror: 1.77
New best_val_rmse: 0.7027

64 steps took 63.0 seconds
Epoch: 0 batch_num: 128
train_rmse_target: 0.6711 train_rmse_stderror: 0.1097 train_kl_div: 1.037
val_rmse_target: 0.6219 val_rmse_stderror: 1.769
New best_val_rmse: 0.6219

64 steps took 63.0 seconds
Epoch: 0 batch_num: 192
train_rmse_target: 0.74 train_rmse_stderror: 0.0948 train_kl_div: 1.216
val_rmse_target: 0.6578 val_rmse_stderror: 1.734
Still best_val_rmse: 0.6219 (from epoch 0)

64 steps took 63.0 seconds
Epoch: 0 batch_num: 256
train_rmse_target: 0.5382 train_rmse_stderror: 0.08356 train_kl_div: 0.6556
val_rmse_target: 0.665 val_rmse_stderror: 1.757
Still best_val_rmse: 0.6219 (from epoch 0)

64 steps took 63.2 seconds
Epoch: 1 batch_num: 37
train_rmse_target: 0.4634 train_rmse_stderror: 0.07315 train_kl_div: 0.383
val_rmse_target: 0.5247 val_rmse_stderr

Some weights of the model checkpoint at /content/clrp-pre-trained/clrp_roberta_large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /content/clrp-pre-trained/clrp_roberta_large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN thi


64 steps took 63.7 seconds
Epoch: 0 batch_num: 64
train_rmse_target: 1.47 train_rmse_stderror: 0.08312 train_kl_div: 3.842
val_rmse_target: 0.7429 val_rmse_stderror: 1.828
New best_val_rmse: 0.7429

64 steps took 63.0 seconds
Epoch: 0 batch_num: 128
train_rmse_target: 0.4918 train_rmse_stderror: 0.05792 train_kl_div: 0.543
val_rmse_target: 0.6752 val_rmse_stderror: 1.762
New best_val_rmse: 0.6752

64 steps took 63.0 seconds
Epoch: 0 batch_num: 192
train_rmse_target: 0.5215 train_rmse_stderror: 0.04248 train_kl_div: 0.59
val_rmse_target: 0.6037 val_rmse_stderror: 1.738
New best_val_rmse: 0.6037

64 steps took 63.0 seconds
Epoch: 0 batch_num: 256
train_rmse_target: 0.5731 train_rmse_stderror: 0.05688 train_kl_div: 0.6388
val_rmse_target: 0.6025 val_rmse_stderror: 1.777
New best_val_rmse: 0.6025

64 steps took 63.1 seconds
Epoch: 1 batch_num: 37
train_rmse_target: 0.5019 train_rmse_stderror: 0.05626 train_kl_div: 0.5377
val_rmse_target: 0.5285 val_rmse_stderror: 1.785
New best_val_rmse: 

Some weights of the model checkpoint at /content/clrp-pre-trained/clrp_roberta_large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /content/clrp-pre-trained/clrp_roberta_large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN thi


64 steps took 63.9 seconds
Epoch: 0 batch_num: 64
train_rmse_target: 0.9152 train_rmse_stderror: 0.2177 train_kl_div: 1.502
val_rmse_target: 0.8615 val_rmse_stderror: 2.043
New best_val_rmse: 0.8615

64 steps took 63.2 seconds
Epoch: 0 batch_num: 128
train_rmse_target: 1.065 train_rmse_stderror: 0.07375 train_kl_div: 2.11
val_rmse_target: 0.9949 val_rmse_stderror: 1.79
Still best_val_rmse: 0.8615 (from epoch 0)

64 steps took 63.2 seconds
Epoch: 0 batch_num: 192
train_rmse_target: 1.223 train_rmse_stderror: 0.09371 train_kl_div: 2.812
val_rmse_target: 0.7802 val_rmse_stderror: 1.787
New best_val_rmse: 0.7802

64 steps took 63.2 seconds
Epoch: 0 batch_num: 256
train_rmse_target: 0.6753 train_rmse_stderror: 0.06491 train_kl_div: 0.8964
val_rmse_target: 0.5743 val_rmse_stderror: 1.807
New best_val_rmse: 0.5743

64 steps took 63.4 seconds
Epoch: 1 batch_num: 37
train_rmse_target: 0.5643 train_rmse_stderror: 0.08372 train_kl_div: 0.753
val_rmse_target: 0.5473 val_rmse_stderror: 1.797
New b

In [24]:
print(list_val_rmse)

[0.49421215826008996, 0.4752608407152704, 0.47807585489870225, 0.4787509217843914, 0.4919866901852834]


In [25]:
#rep = MemReporter(model)
#rep.report()

In [26]:
#rep = MemReporter(model.roberta)
#rep.report()

In [27]:
#gpuinfo()

In [28]:
#del model
#del optimizer 
#del train_loader
#del val_loader
#del scheduler 
#del list_val_rmse
#del train_indices
#del val_indices
#del tokenizer
#torch.cuda.empty_cache()
#gpuinfo()

# upload models

In [29]:
%cd
!mkdir .kaggle
!mkdir /content/model
!cp /content/drive/MyDrive/Colab_Files/kaggle-api/kaggle.json .kaggle/

!cp -r /content/model_1.pth /content/model/model_1.pth
!cp -r /content/model_2.pth /content/model/model_2.pth
!cp -r /content/model_3.pth /content/model/model_3.pth
!cp -r /content/model_4.pth /content/model/model_4.pth
!cp -r /content/model_5.pth /content/model/model_5.pth
!cp -r /content/TFIDF_1.pkl /content/model/TFIDF_1.pkl
!cp -r /content/TFIDF_2.pkl /content/model/TFIDF_2.pkl
!cp -r /content/TFIDF_3.pkl /content/model/TFIDF_3.pkl
!cp -r /content/TFIDF_4.pkl /content/model/TFIDF_4.pkl
!cp -r /content/TFIDF_5.pkl /content/model/TFIDF_5.pkl

/root


In [30]:
def dataset_upload():
    import json
    from kaggle.api.kaggle_api_extended import KaggleApi

    id = f'{USERID}/{EX_NO}'

    dataset_metadata = {}
    dataset_metadata['id'] = id
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = f'{EX_NO}'

    with open(UPLOAD_DIR / 'dataset-metadata.json', 'w') as f:
        json.dump(dataset_metadata, f, indent=4)

    api = KaggleApi()
    api.authenticate()

    # データセットがない場合
    if f'{USERID}/{EX_NO}' not in [str(d) for d in api.dataset_list(user=USERID, search=f'"{EX_NO}"')]:
        api.dataset_create_new(folder=UPLOAD_DIR,
                               convert_to_csv=False,
                               dir_mode='skip')
    # データセットがある場合
    else:
        api.dataset_create_version(folder=UPLOAD_DIR,
                                   version_notes='update',
                                   convert_to_csv=False,
                                   delete_old_versions=True,
                                   dir_mode='skip')
dataset_upload()



  1%|          | 8.30M/1.33G [00:00<00:16, 85.6MB/s]

Starting upload for file model_4.pth


100%|██████████| 1.33G/1.33G [00:12<00:00, 113MB/s]
  0%|          | 0.00/546k [00:00<?, ?B/s]

Upload successful: model_4.pth (1GB)
Starting upload for file TFIDF_5.pkl


100%|██████████| 546k/546k [00:00<00:00, 2.21MB/s]
  0%|          | 0.00/542k [00:00<?, ?B/s]

...

Upload successful: TFIDF_4.pkl (543KB)
Starting upload for file model_3.pth


100%|██████████| 1.33G/1.33G [00:15<00:00, 90.2MB/s]
  1%|          | 6.87M/1.33G [00:00<00:23, 61.3MB/s]

Upload successful: model_3.pth (1GB)
Starting upload for file model_2.pth


100%|██████████| 1.33G/1.33G [00:20<00:00, 68.3MB/s]


Upload successful: model_2.pth (1GB)


In [31]:
# validation再実行_予測結果取得

all_predictions = np.zeros(len(train_kf_df)) # 推論結果について、「fold　× 推論df」のzero行列で枠を作る

for fold_ in sorted(train_kf_df['kfold'].unique()):
    model_path = UPLOAD_DIR/f"model_{fold_ + 1}.pth" # 対応するモデルを読む
    tfidf_path = UPLOAD_DIR/f"TFIDF_{fold_ + 1}.pkl"

    with open(tfidf_path, 'rb') as f:
        tfidf = pickle.load(f)

    print(f"\nUsing {model_path}")

    val_idx = train_kf_df['kfold'] == fold_
    val_df = train_kf_df[val_idx]
    val_dataset = LitDataset(val_df, tfidf, inference_only=True) # TestのDataset(何で、もう一回作るのだろう？)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,
                          drop_last=False, shuffle=False, num_workers=2) # TestのDataLoader

    model = LitModel()
    model.load_state_dict(torch.load(model_path))    # 対応するモデルから、重みを読み込む
    model.to(DEVICE) # モデルをDEVICEへぶち込む

    all_predictions[val_idx] = predict(model, val_loader) # 推論結果行列の対象列に、推論結果を入力(以後、繰り返し)

    del model
    gc.collect()



Using /content/model/model_1.pth


Some weights of the model checkpoint at /content/clrp-pre-trained/clrp_roberta_large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /content/clrp-pre-trained/clrp_roberta_large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN thi

NameError: ignored

In [ ]:
train_kf_df['pred'] = all_predictions
train_kf_df['diff_sq'] = (train_kf_df['target'] - train_kf_df['pred'])**2

In [ ]:
train_kf_df.plot(kind='scatter', x='target', y='diff_sq')

In [ ]:
# 二乗誤差が2.0を超えるレコード
thr_ = 2.0 
train_kf_df[train_kf_df['diff_sq'] > thr_]

In [ ]:
# 二乗誤差が2.0を超える文章
thr_ = 2.0 
tmp_df = train_kf_df[train_kf_df['diff_sq'] > thr_].copy()
for i in tmp_df.index:
  print(tmp_df.loc[i].target)
  #print(tmp_df.loc[i].standard_error)
  print(tmp_df.loc[i].pred)
  print(tmp_df.loc[i].excerpt)
  print('--------------------------')

In [ ]:
%env MODEL_OUT_DIR '/content/drive/MyDrive/Colab_Files/kaggle/commonlit/98_model_inf/055-054-train-03'
!mkdir -p $MODEL_OUT_DIR
!cp -r /content/model/ $MODEL_OUT_DIR

In [ ]:
!cp -r /content/model_1.pth /content/model/model_1.pth
!cp -r /content/model_2.pth /content/model/model_2.pth
!cp -r /content/model_3.pth /content/model/model_3.pth
!cp -r /content/model_4.pth /content/model/model_4.pth
!cp -r /content/model_5.pth /content/model/model_5.pth
!cp -r /content/TFIDF_1.pkl /content/model/TFIDF_1.pkl
!cp -r /content/TFIDF_2.pkl /content/model/TFIDF_2.pkl
!cp -r /content/TFIDF_3.pkl /content/model/TFIDF_3.pkl
!cp -r /content/TFIDF_4.pkl /content/model/TFIDF_4.pkl
!cp -r /content/TFIDF_5.pkl /content/model/TFIDF_5.pkl

In [ ]:
model_path_out = Path('/content/model/')

In [ ]:
for i in list(model_path_out.iterdir()):
  print(i)

In [ ]:
import shutil
tgdir = Path('/content/drive/MyDrive/Colab_Files/kaggle/commonlit/98_model_inf/055-054')

for file_ in list(model_path_out.iterdir()):
  shutil.copy(file_, tgdir)